In [1]:
class Guest:
    '''
    A representation of a guest in the invitation CSP
    '''

    NOT_INVITED = 0
    INVITED = 1
    UNDECIDED = 2

    def __init__(self, name, invited = UNDECIDED):
        '''
        Constructor for the Guest object
        :param name: the name of a guest
        :param invited: invitation status for a guest
        '''
        self.name = name
        self.invited = invited

    # some checks for invitation status
    def is_invited(self):
        return self.invited == Guest.INVITED

    def is_not_invited(self):
        return self.invited == Guest.NOT_INVITED

    def is_undecided(self):
        return self.invited == Guest.UNDECIDED

    def __eq__(self, other):
        '''
        :param other: a Guest node to compare with
        :return: True if self and other have the same name
        '''
        return self.name == other.name

In [2]:
import copy

In [3]:
class InvitationNode:
    '''
        The class represents the node in a search tree for the invitation csp
    '''

    def __init__(self):
        '''
             A constructor that represent the state of the csp problem at a particular node
        '''
        # Normally one would not hard code data in to a program, but sometimes simplicity is
        # the easiest way to do thing
        # Here you should add all the guests that might be relevant for your party
        # according to the pattern on the next lines
        self.anne = Guest("Anne")
        self.kari = Guest("Kari")
        self.randi = Guest("Randi")
        self.liv = Guest("Liv")
        self.gro = Guest("Gro")
        self.ola = Guest("Ola")
        self.jan = Guest("Jan")
        self.henning = Guest("Henning")
        self.knut = Guest("Knut")
        self.ivar = Guest("Ivar")

        # You also need to make a list of all these potential guests
        self.assignment = [self.anne, self.kari, self.randi, self.liv, self.gro, self.ola, self.jan, self.henning, self.knut, self.ivar]
        self.women = [self.anne, self.kari, self.randi, self.liv, self.gro]
        self.men = [self.ola, self.jan, self.henning, self.knut, self.ivar]

    def get_neighbours(self):
        '''
            Finds the consistent children of a node in the search tree
        :return: a list of children
        '''
        result = []

        n = self.copy_and_add_assignment(Guest.INVITED)
        # generate a child where the next undecided guest is tested for invitation
        if n is not None and n.is_consistent():
            # if there is such a guest and i the invitation assignment is consistent then append n
            result.append(n)

        n = self.copy_and_add_assignment(Guest.NOT_INVITED)
        # generate a child where the next undecided guest is tested for not invitation
        if n is not None and n.is_consistent():
            # if there is such a guest and i the invitation assignment is consistent then append n
            result.append(n)

        return result

    def copy_and_add_assignment(self,invite):
        '''
        Copys a node and adds one invitation status to one potential guest
        :param invite: the invitation status to set
        :return: a node with invitation set for one more guest
        '''
        new_node = copy.deepcopy(self)
        # copy the node
        for guest in new_node.assignment:
            # find a guest that is undecided in self's assignment
            if guest.is_undecided():
                # set this particular guests invitation status to 'invite'
                guest.invited = invite
                return new_node
        return None

    def is_consistent(self):
        '''
        The function that checks if assignments are consistent
        :return: True if an assignment is consistent with the constraints otherwise False
        '''
        val = True
        # Here we add all the constraints
        # This should not normally be hardcoded in more serious CSP programs.
        #
        # The idea is that only relevant constraints should be checked.
        # To be relevant the node needs to have an assignment that is not UNDECIDED for any of the guests in
        # the constraint. We check this by using the not_relevant_constraint('list of guest objects') function.
        # The parameter to that function should be the guests involved in the constraint
        # So every line below here in the function (except the two last ones) should be of the form
        # val = val and (self.not_relevant_constraint('list of guests in constraint') or ('constraint on one or more guests'))

        # Here you need to fill in the constraints according to the given pattern

        #1
        val = val and(self.not_relevant_constraint([self.anne, self.ola]) or
                      (self.anne.is_not_invited() or self.ola.is_not_invited()))
        # 2
        val = val and (self.not_relevant_constraint([self.kari, self.jan]) or
                       ((self.kari.is_not_invited() or self.jan.is_invited()) and
                        (self.kari.is_invited() or self.jan.is_not_invited())))
        # 3
        val = val and (self.not_relevant_constraint([self.ola, self.ivar, self.henning, self.knut]) or
                       (((self.ola.is_not_invited() or self.ivar.is_not_invited()) or
                         (self.henning.is_not_invited() or self.knut.is_not_invited()))))
        # 4
        val = val and (self.not_relevant_constraint([self.randi, self.liv, self.ivar]) or
                       (self.ivar.is_not_invited() or (self.randi.is_not_invited() and self.liv.is_not_invited())))
        # 5
        val = val and (self.not_relevant_constraint([self.gro, self.henning, self.kari]) or
                       ((self.henning.is_not_invited() and self.kari.is_not_invited()) or self.gro.is_not_invited()))
        # 6
        val = val and (self.not_relevant_constraint([self.gro, self.knut, self.jan]) or
                       ((self.gro.is_not_invited() or self.knut.is_not_invited()) or self.jan.is_not_invited()))
        # 7
        val = val and (self.not_relevant_constraint([self.gro, self.anne, self.knut, self.ola]) or
                       (((self.gro.is_not_invited() or self.anne.is_not_invited()) or
                         (self.knut.is_not_invited() and self.ola.is_not_invited()))))
        # 8
        val = val and (self.not_relevant_constraint([self.ivar, self.henning, self.liv]) or
                       (self.ivar.is_invited() or (self.henning.is_invited() and self.liv.is_invited())))

        val = val and self.is_ok_guest_count()
        return val

    def is_ok_guest_count(self):
        count_invited = 0
        count_undecided = 0
        women_invited = 0
        women_undecided = 0
        men_invited = 0
        men_undecided = 0
        for guest in self.assignment:
            if guest.is_invited():
                if guest in self.women:
                    women_invited = women_invited + 1
                elif guest in self.men:
                    men_invited = men_invited +1
                count_invited = count_invited+1
            elif guest.is_undecided():
                if guest in self.women:
                    women_undecided = women_undecided + 1
                elif guest in self.men:
                    men_undecided = men_undecided +1
                count_undecided = count_undecided+1
        # wants exact 6 guests, 3 women and 3 men
        return (count_invited <= 6 and count_invited + count_undecided >= 6 and \
            women_invited <= 3 and women_invited + women_undecided >= 3 and \
            men_invited <= 3 and men_invited + men_undecided >= 3)



    def not_relevant_constraint(self,  constraint_guests):
        '''
        Local function that checks that all guests in a constraint are not UNDECIDED
        :param constraint_guests: the guests involved in a constraint
        :return: True if some guests in the list is UNDECIDED
        '''
        val = False
        for g in constraint_guests:
            if g.is_undecided():
                val = True
        return val

    def is_goal(self):
        '''
        Checks if we are at a goal/leaf in the search tree
        :return: True if all guests have been DECIDED
        '''
        for guest in self.assignment:
            if guest.is_undecided():
                return False
        return True

    def __str__(self):
        '''
        :return: A string representation of an Invitation Node
        '''
        str = ''
        for g in self.assignment:
            if g.invited == Guest.INVITED:
                status = "Invited"
            elif g.invited == Guest.NOT_INVITED:
                status = "Not invited"
            elif g.invited == Guest.UNDECIDED:
                status == "Undecided"
            str = str + '{:6} {}\n'.format(g.name,status)
        return str


In [4]:
class InvitationSolver:

    '''
        The class contains for a CSP solver that ims to find invitations of a set of potential guests to a party
        according to some constraints.
        The class contains the search strategy, which is basically a depth first search, but continued so it finds
        all solutions
    '''


    def __init__(self):
        '''
        Constructor that creates an empty search frontier for depth first search
        and then adds one node to the search tree that contains an empty assigmnent of guests
        self.count is just use for counting the nodes of the tree that is generated during search
        Notice the use of append() and pop() to realise a queue frontier
        '''
        self.frontier = []
        self.frontier.append(InvitationNode())
        self.count = 1

    def run_search(self):
        '''
        runs a depth-first-search on the search tree.
        Notice: pop() in python removes the last element from a list
        :return: a solution of the search in the form of an InvitationNode
        '''
        while len(self.frontier) != 0:
            node = self.frontier.pop()
            if node.is_goal():
                return node
            self.update_frontier(node)
        return None

    def update_frontier(self, node):
        '''
        Finds new nodes to be added to the search frontier
        :param node: the node that we want to find children of
        :return: void
        '''
        new_nodes = node.get_neighbours()
        for node in new_nodes:
            self.count = self.count+1
            self.frontier.append(node)

if __name__ == "__main__":
    '''
        runs the invitation solver
    '''
    s = InvitationSolver()
    # initialize it
    sol = s.run_search()
    # run the first search
    count = 0
    while sol is not None:
        count = count+1
        print(sol)
        # print a solution
        sol = s.run_search()
        # find the next solution in the search tree

    print(count)
    # print the number of nodes investigated


Anne   Not invited
Kari   Invited
Randi  Invited
Liv    Invited
Gro    Not invited
Ola    Not invited
Jan    Invited
Henning Invited
Knut   Invited
Ivar   Not invited

Anne   Not invited
Kari   Invited
Randi  Invited
Liv    Invited
Gro    Not invited
Ola    Invited
Jan    Invited
Henning Invited
Knut   Not invited
Ivar   Not invited

Anne   Invited
Kari   Invited
Randi  Not invited
Liv    Invited
Gro    Not invited
Ola    Not invited
Jan    Invited
Henning Invited
Knut   Invited
Ivar   Not invited

3
